In [15]:
"""
    Classificação de digitos escritos à mão com o uso da CNN
    Atividade do curso 'Deep Learning com Python de A a Z - O Curso Completo'
    Marcio Salmazo Ramos
"""

import matplotlib.pyplot as plt
from keras.datasets import mnist  # Importação da base de dados
from keras.models import Sequential  # Modelo sequencial
from keras.layers import Dense, Flatten  # 3ª e 4ª etapa da CNN
from keras.layers import Conv2D, MaxPooling2D  # 1ª e 2ª etapa da CNN
from keras import utils


<p>Realizando o tratamento de dados no Dataset</p>

In [6]:
'''
    Carregar a base de dados, já com a separação pré-definida para teste e treinamento
    x -> atributos previsores
    y -> classes da base
'''
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
sizeXTRAIN = xTrain.shape
sizeXTEST = xTest.shape
sizeYTRAIN = yTrain.shape
sizeYTEST = yTest.shape

print('Shape dos atributos previsores de teste', sizeXTEST)
print('Shape dos atributos previsores de treinamento', sizeXTRAIN)
print('Shape das classes de teste', sizeYTEST)
print('Shape das classes de treinamento', sizeYTRAIN)

'''
    Existem 60000 imagens para treinamento 28x28 px
    Existem 10000 imagens para teste 28x28 px
    Existem 10000 valores como resposta para as imagens de teste
    Existem 60000 valores como resposta para as imagens de treinamento
'''

Shape dos atributos previsores de teste (10000, 28, 28)
Shape dos atributos previsores de treinamento (60000, 28, 28)
Shape das classes de teste (10000,)
Shape das classes de treinamento (60000,)


'\nExistem 60000 imagens para treinamento 28x28 px\nExistem 10000 imagens para teste 28x28 px\nExistem 10000 valores como resposta para as imagens de teste\nExistem 60000 valores como resposta para as imagens de treinamento\n'

In [23]:
'''
    É necessário realizar uma transformação dos dados a fim de que o tensorflow
    consiga analizá-los. Neste caso, a dimensionalidade de cada imagem será diminuida,
    ou seja, apenas uma camada RGB será utilizada 
'''
trainForecasters = xTrain.reshape(xTrain.shape[0], 28, 28, 1)
testForecasters = xTest.reshape(xTest.shape[0], 28, 28, 1)

'''
    Alterando o tipo dos dados para float32 a fim de aplicar 
    a normalização futuramente
'''

trainForecasters = trainForecasters.astype('float32')
testForecasters = testForecasters.astype('float32')

In [24]:
'''
    Realizando a normalização (min/max normalization) a fim de que os valores dos pixels estejam
    entre 0 e 1, tornando o processamento mais eficiente
    obs: 255 é o valor máximo do pixel
'''

trainForecasters /= 255
testForecasters /= 255

In [25]:
'''
    Criação de dummy variables a fim de criar um encoder para 
    os diferentes valores.
    Uma variável Dummy é aquela que toma o valor de "zero" ou "um" 
    indicando a ausência ou presença de qualidades ou atributos. 
    Essas variáveis são usadas como dispositivos para classificar 
    dados em categorias mutuamente exclusivas.

    Esta etapa se faz necessária, uma vez que o problema exige 
    classificação em mais de duas classes

    Com método utils.to_categorical no keras, um array de valores
    inteiro que representem diferentes classes pode ser convertido em
    um array ou matriz de valores binários que representem os inteiros 
    no array original, exemplo:

    Arr = <1,2,3,4>
    Mat = utils.to_categorical(Arr, 4)

    Mat = <
        <0,0,0,1>
        <0,0,1,0>
        <0,0,1,1>
        <0,1,0,0>
    >
'''
trainClass = utils.to_categorical(yTrain, 10)
testClass = utils.to_categorical(yTest, 10)


<p>Aplicando a rede neural densa</p>

In [26]:
'''
    Aplicando o operador de convolução
    A variável 'classifier recebe o modelo sequencial, ou seja: 
    permite inserir camadas de uma rede neural em série, onde o output da primeira 
    camada serve como input da segunda, e assim por diante.'

    A primeira camada (referência à imagem) contém o resultado do operador de 
    convolução. Neste caso, é definido que serão utilizados 32 filtros ou
    detectores de características para cada convolução e o kernal utilizado
    será 3x3. Por fim, a função de ativação para cada convolução será o Relu
'''

classifier = Sequential()
classifier.add(Conv2D(32, (3,3), activation = 'relu'))

In [27]:
'''
    A segunda camada será composta pelo operador de pooling
    aplicado ao resultado das convoluções anteriores
    É preciso definir o tamanho da máscara para o processo
    de pooling, neste caso será usado uma matriz 2x2
'''

classifier.add(MaxPooling2D(pool_size = (2,2)))

In [28]:
'''
    A terceira etapa se dá apenas pelo achatamento da matriz
    obtido após o processo de pooling na etapa anterior
'''

classifier.add(Flatten())

In [29]:
'''
    A quarta etapa do processo se dá pela adição da rede neural densa
    tendo como entrada os valores provenientes do vetor obtido na 
    etapa anterior.

    Nesta implementação, a rede neural conta com apenas uma camada
    intermediária contendo 128 neurônios, e para cada um de seus 
    neurônios a função de ativação será a função Relu. Além disso,
    também é definida a camada de saída (contendo 10 saídas) e tem 
    a função 'softmax' como função de ativação 
'''

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 10, activation = 'softmax'))

In [30]:
'''
    A rede criada será compilada 
'''
classifier.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
classifier.fit(trainForecasters, trainClass, batch_size = 128, epochs = 5, validation_data = (testForecasters, testClass))


Epoch 1/5


469/469 [==============================] - 8s 16ms/step - loss: 0.2314 - accuracy: 0.9328 - val_loss: 0.0917 - val_accuracy: 0.9732
Epoch 2/5
469/469 [==============================] - 8s 17ms/step - loss: 0.0737 - accuracy: 0.9784 - val_loss: 0.0578 - val_accuracy: 0.9815
Epoch 3/5
469/469 [==============================] - 8s 16ms/step - loss: 0.0483 - accuracy: 0.9854 - val_loss: 0.0439 - val_accuracy: 0.9839
Epoch 4/5
469/469 [==============================] - 8s 18ms/step - loss: 0.0346 - accuracy: 0.9896 - val_loss: 0.0411 - val_accuracy: 0.9848
Epoch 5/5
469/469 [==============================] - 8s 17ms/step - loss: 0.0273 - accuracy: 0.9916 - val_loss: 0.0397 - val_accuracy: 0.9861
